**Diplomado en Inteligencia Artificial y Aprendizaje Profundo**

# Modelo Lineal de Clasificación  con JAX

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Función de Predicción](#Función-de-predicción)
* [Función de Pérdida. Entropía cruzada](#Función-de-pérdida.-Entropía-cruzada)
* [Ejemplo. Datos de Juguete](#Ejemplo.-Datos-de-Juguete)
* [Gradiente](#Gradiente)
* [Entrenamiento del modelo](#Entrenamiento-del-modelo)


## Introducción

Con su versión actualizada de [Autograd](https://github.com/hips/autograd), [JAX](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html) puede diferenciar automáticamente el código nativo de Python y NumPy. Puede derivarse través de un gran subconjunto de características de Python, incluidos bucles, ifs, recursión y clousures, e incluso puede tomar derivadas de derivadas de derivadas. Admite la diferenciación tanto en modo inverso como en modo directo, y los dos pueden componerse arbitrariamente en cualquier orden.

Lo nuevo es que JAX usa [XLA](https://www.tensorflow.org/xla) para compilar y ejecutar su código NumPy en aceleradores, como GPU y TPU. La compilación ocurre de forma predeterminada, con las llamadas de la biblioteca compiladas y ejecutadas justo a tiempo. Pero JAX incluso le permite compilar justo a tiempo sus propias funciones de Python en núcleos optimizados para XLA utilizando una API de una función. La compilación y la diferenciación automática se pueden componer de forma arbitraria, por lo que puede expresar algoritmos sofisticados y obtener el máximo rendimiento sin tener que abandonar Python.

In [ ]:
# !pip install --upgrade jax jaxlib 

from __future__ import print_function
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
key = random.PRNGKey(0)
# Current convention is to import original numpy as "onp"
import numpy as onp
import itertools


#import random
#import jax


## Función de predicción

In [ ]:
def sigmoid(x):
    return 0.5*(np.tanh(x/2)+1)
# more stable than  1.0/(1+np.exp(-x))

# outputs probability of a label being true
def predict(W,b,inputs):
    return sigmoid(np.dot(inputs,W)+b)

## Función de pérdida. Entropía cruzada

In [ ]:

# training loss: -log likelihood of trainig examples
def loss(W,b,x,y):
    preds = predict(W,b,x)
    label_probs = preds*y + (1-preds)*(1-y)
    return -np.sum(np.log(label_probs))

# initialize coefficients
key, W_key, b_key = random.split(key,3)
W = random.normal(key, (3,))
b = random.normal(key,())

## Ejemplo. Datos de Juguete

In [ ]:
# Build a toy dataset
inputs = np.array([[0.52, 1.12,  0.77],
                   [0.88, -1.08, 0.15],
                   [0.52, 0.06, -1.30],
                   [0.74, -2.49, 1.39]])
targets = np.array([True, True, False, True])

## Gradiente

Usaremos la funcion *grad* con sus argumentos  para diferenciar la función con respecto a sus parámetros ṕosicionales

In [ ]:
# compile with jit
# argsnums define positional params to derive with respect to
grad_loss = jit(grad(loss,argnums=(0,1)))

In [ ]:
W_grad, b_grad = grad_loss(W,b,inputs, targets)
print("W_grad = ", W_grad)
print("b_grad = ", b_grad)

## Entrenamiento del modelo

In [ ]:
# train function
def train(W,b,x,y, lr= 0.12):
    gradient = grad_loss(W,b,inputs,targets) 
    W_grad, b_grad = grad_loss(W,b,inputs,targets)
    W -= W_grad*lr
    b -= b_grad*lr
    return(W,b)

In [ ]:
#    
weights, biases = [], []
train_loss= []
epochs = 20

train_loss.append(loss(W,b,inputs,targets))

for epoch in range(epochs):
    W,b = train(W,b,inputs, targets)
    weights.append(W)
    biases.append(b)
    losss = loss(W,b,inputs,targets)
    train_loss.append(losss)
    print(f"Epoch {epoch}: train loss {losss}")

In [ ]:
print('weights')
for weight in weights:
    print(weight)
print('biases')
for bias in biases:
    print(bias)

In [ ]:
print(grad(loss)(W,b,inputs,targets))

### Calculando el valor de la función y el gradiente con value_and_grad

In [ ]:
from jax import value_and_grad
loss_val, Wb_grad = value_and_grad(loss,(0,1))(W,b,inputs, targets)
print('loss value: ', loss_val)
print('gradient value: ', Wb_grad)

- [Regresar al inicio](#Contenido)